In [1]:
north_pole = (90,0)
weight_limit = 1000
sleigh_weight = 10

In [2]:
import pandas as pd
import numpy as np
from haversine import haversine
from operator import itemgetter

In [3]:
def weighted_trip_length(stops, weights): 
    tuples = [tuple(x) for x in stops.values]
    # adding the last trip back to north pole, with just the sleigh weight
    tuples.append(north_pole)
    weights.append(sleigh_weight)
    
    dist = 0.0
    prev_stop = north_pole
    prev_weight = sum(weights)
    for i in range(len(tuples)):        
        dist = dist + haversine(tuples[i], prev_stop) * prev_weight
        prev_stop = tuples[i]   
        prev_weight = prev_weight - weights[i]
    return dist

In [4]:
def weighted_reindeer_weariness(all_trips):
    uniq_trips = all_trips.TripId.unique()
    
    if any(all_trips.groupby('TripId').Weight.sum() > weight_limit):
        raise Exception("One of the sleighs over weight limit!")
 
    dist = 0
    for t in uniq_trips:
        this_trip = all_trips[all_trips.TripId==t]
        dist = dist + weighted_trip_length(this_trip[['Latitude','Longitude']], this_trip.Weight.tolist())
    
    return dist    

In [5]:
def get_worst_gift(trip):
    gifts = trip['GiftId'].values
    trip_wo_gifts = [trip.loc[trip['GiftId'] != gift] for gift in gifts]
    weight_wo_gifts = [weighted_reindeer_weariness(t) for t in trip_wo_gifts]
    return min(zip(gifts, weight_wo_gifts), key=itemgetter(1))[0]

In [6]:
def get_worst_gifts(all_trips):
    uniq_trips = all_trips.TripId.unique()
    return [get_worst_gift(all_trips[all_trips.TripId==t]) for t in uniq_trips]

In [7]:
gifts = pd.read_csv('./gifts.csv')
sample_sub = pd.read_csv('./sample_submission.csv')

In [8]:
def wrw_for_two_trips(trips, trip1, trip2):
    return weighted_reindeer_weariness(trips.loc[trips['TripId'].isin([trip1, trip2])])

In [9]:
def get_trip(index):
    all_trips[all_trips.TripId==index]

In [10]:
def get_good_bad_trips(trips):
    uniq_trip_names = trips.TripId.unique()
    uniq_trips = [trips[trips.TripId==t] for t in uniq_trip_names]
    bad_trips = [t for t in uniq_trips if not weights_low_enough(t)]
    good_trips = [t for t in uniq_trips if weights_low_enough(t)]
    return [good_trips, bad_trips]

In [11]:
def weights_low_enough(trips):
    try:
        weighted_reindeer_weariness(trips)
        return True
    except Exception:
        return False

In [12]:
def halve(trip):
    by_weight = trip.sort_values('Weight')
    tripa = by_weight.iloc[::2]
    tripb = by_weight.iloc[1::2]
    return (tripa, tripb)

In [13]:
def divide(trip):
    if weights_low_enough(trip):
        return trip
    else:
        tripa, tripb = halve(trip)
        orig_id = str(trip['TripId'].values[0])
        tripa.loc[:, 'TripId'] = orig_id+'a'
        tripb.loc[:, 'TripId'] = orig_id+'b'
        return pd.concat([divide(tripa), divide(tripb)])

In [14]:
def shrink_clusters(trips):
    good_trips, bad_trips = get_good_bad_trips(trips)
    return pd.concat(good_trips + [divide(b) for b in bad_trips])

In [15]:
def cleanup(trips):
    uniq_trip_names = trips.TripId.unique()
    maxint = max([n for n in uniq_trip_names if type(n)==int])
    string_names = [n for n in uniq_trip_names if type(n)==str]
    new_int_names = [maxint + string_names.index(s) for s in string_names]
    d_int_names = dict(zip(string_names, new_int_names))
    
    new_trips = trips.copy()
    #new_trips.loc[new_trips.TripId.isin(string_names)].map()
    new_trips.replace({"TripId": d_int_names}, inplace=True)
    return new_trips

In [16]:
import sklearn.cluster

In [31]:
km = sklearn.cluster.KMeans(n_clusters=5000)

In [32]:
x = km.fit_predict(gifts[['Latitude', 'Longitude']].values)

In [33]:
x

array([307, 752, 760, ..., 678, 316,  36], dtype=int32)

In [34]:
result = gifts.copy()

In [35]:
result['TripId'] = x

In [36]:
result

,GiftId,Latitude,Longitude,Weight,TripId
0,1,16.345769,6.303545,1.000000,307
1,2,12.494749,28.626396,15.524480,752
2,3,27.794615,60.032495,8.058499,760
3,4,44.426992,110.114216,1.000000,865
4,5,-69.854088,87.946878,25.088892,356
5,6,53.567970,-71.359308,38.000151,959
6,7,12.902584,79.966949,44.206616,21
7,8,-6.291099,-64.891751,1.000000,756
8,9,-2.685316,111.089758,1.000000,688
9,10,38.428862,101.973671,35.701311,845


In [37]:
weighted_reindeer_weariness(result)

Exception: One of the sleighs over weight limit!

In [38]:
shrunk_result = shrink_clusters(result)

In [39]:
weighted_reindeer_weariness(shrunk_result)

24943495753.746822

In [267]:
len(shrunk_result.TripId.unique())

4500

In [268]:
clean_result = cleanup(shrunk_result)

ValueError: max() arg is an empty sequence

In [277]:
result.drop(['Longitude', 'Latitude', "Weight"], 1).set_index('GiftId').to_csv('sub9(146e8).csv')